# Training Deep Kernel Learning with a DenseNet on CIFAR10/100

In this notebook, we'll demonstrate the steps necessary to train a medium sized DenseNet (https://arxiv.org/abs/1608.06993) on either of two popularly used benchmark dataset in computer vision (CIFAR10 and CIFAR100). We'll be training the DKL model entirely end to end using the standard 300 Epoch training schedule and SGD.

This notebook is largely for tutorial purposes. If your goal is just to get (for example) a trained DKL + CIFAR100 model, we __recommend__ that you move this code to a simple python script and run that, rather than training directly out of a python notebook. We find that training is just a bit faster out of a python notebook.

Furthermore, because this notebook involves training an actually reasonably large neural network, it is __strongly recommended__ that you have a decent GPU available for this, as with all large deep learning models.

In [1]:
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F
from torch import nn
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import gpytorch

## Set up data augmentation

The first thing we'll do is set up some data augmentation transformations to use during training, as well as some basic normalization to use during both training and testing. We'll use random crops and flips to train the model, and do basic normalization at both training time and test time. To accomplish these transformations, we use standard `torchvision` transforms.

In [2]:
normalize = transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
aug_trans = [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()]
common_trans = [transforms.ToTensor(), normalize]
train_compose = transforms.Compose(aug_trans + common_trans)
test_compose = transforms.Compose(common_trans)

## Create DataLoaders

Next, we create dataloaders for the selected dataset using the built in torchvision datasets. The cell below will download either the cifar10 or cifar100 dataset, depending on which choice is made. The default here is cifar10, however training is just as fast on either dataset.

After downloading the datasets, we create standard `torch.utils.data.DataLoader`s for each dataset that we will be using to get minibatches of augmented data.

In [3]:
dataset = 'cifar10'

if dataset == 'cifar10':
    d_func = dset.CIFAR10
    train_set = dset.CIFAR10('data', train=True, transform=train_compose, download=True)
    test_set = dset.CIFAR10('data', train=False, transform=test_compose)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=3, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=3, pin_memory=True)
    num_classes = 10
elif dataset == 'cifar100':
    d_func = dset.CIFAR100
    train_set = dset.CIFAR100('data', train=True, transform=train_compose, download=True)
    test_set = dset.CIFAR100('data', train=False, transform=test_compose)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=3, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=3, pin_memory=True)
    num_classes = 100
else:
    raise RuntimeError('dataset must be one of "cifar100" or "cifar10"')

Files already downloaded and verified


## Creating the DenseNet model

With the data loaded, we can move on to defining our DKL model. A DKL model consists of three components: the neural network, the Gaussian process layer used after the neural network, and the Softmax likelihood.

The first step is defining the neural network architecture. To do this, we use a slightly modified version of the DenseNet available in the standard PyTorch package. Specifically, we modify it to remove the softmax layer, since we'll only be needing the final features extracted from the neural network.

In [5]:
from densenet import DenseNet

class DenseNetFeatureExtractor(DenseNet):
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=self.avgpool_size).view(features.size(0), -1)
        return out

feature_extractor = DenseNetFeatureExtractor(block_config=(6, 6, 6), num_classes=num_classes).cuda()
num_features = feature_extractor.classifier.in_features

## Creating the GP Layer

In the next cell, we create the layer of Gaussian process models that are called after the neural network. In this case, we'll be using one GP per feature, as in the SV-DKL paper. The outputs of these Gaussian processes will the be mixed in the softmax likelihood.

In [6]:
class GaussianProcessLayer(gpytorch.models.AdditiveGridInducingVariationalGP):
    def __init__(self, n_features, grid_bounds=(-10., 10.), grid_size=128):
        super(GaussianProcessLayer, self).__init__(grid_size=grid_size, grid_bounds=[grid_bounds],
                                              n_components=n_features, mixing_params=False, sum_output=False)
        self.covar_module = gpytorch.kernels.RBFKernel()
        self.mean_module = gpytorch.means.ConstantMean()
        self.grid_bounds = grid_bounds

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

## Creating the DKL Model

With both the DenseNet feature extractor and GP layer defined, we can put them together in a single module that simply calls one and then the other, much like building any Sequential neural network in PyTorch. This completes defines our DKL model.

In [7]:
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, n_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GaussianProcessLayer(n_features=n_features, grid_bounds=grid_bounds)
        self.grid_bounds = grid_bounds
        self.n_features = n_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features.unsqueeze(-1))
        return res

model = DKLModel(feature_extractor, n_features=num_features).cuda()
likelihood = gpytorch.likelihoods.SoftmaxLikelihood(n_features=model.n_features, n_classes=num_classes).cuda()

## Defining Training and Testing Code

Next, we define the basic optimization loop and testing code. This code is entirely analogous to the standard PyTorch training loop. We create a `torch.optim.SGD` optimizer with the parameters of the neural network on which we apply the standard amount of weight decay suggested from the paper, the parameters of the Gaussian process (from which we omit weight decay, as L2 regualrization on top of variational inference is not necessary), and the mixing parameters of the Softmax likelihood. 

We use the standard learning rate schedule from the paper, where we decrease the learning rate by a factor of ten 50% of the way through training, and again at 75% of the way through training.

In [8]:
n_epochs = 300
lr = 0.1
optimizer = SGD([
    {'params': model.feature_extractor.parameters(), 'lr': lr},
    {'params': model.gp_layer.parameters(), 'lr': lr},
    {'params': likelihood.parameters(), 'lr': lr},
], lr=lr, momentum=0.9, nesterov=True, weight_decay=0)
scheduler = MultiStepLR(optimizer, milestones=[0.5 * n_epochs, 0.75 * n_epochs], gamma=0.1)

def train(epoch, lr=0.1):
    model.train()
    likelihood.train()

    train_loss = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = -model.gp_layer.marginal_log_likelihood(likelihood, output, target, n_data=len(train_loader.dataset))
        loss.backward()
        optimizer.step()
        print('Train Epoch: %d [%03d/%03d], Loss: %.6f' % (epoch, batch_idx + 1, len(train_loader), loss.item()))

def test():
    model.eval()
    likelihood.eval()

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            output = likelihood(model(data))
            pred = output.argmax()
            correct += pred.eq(target.view_as(pred)).data.cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Train the model

We are now ready to train the model. At the end of each Epoch we report the current test loss and accuracy, and we save a checkpoint model out to a file.

In [ ]:
for epoch in range(1, n_epochs + 1):
    scheduler.step()
    with gpytorch.settings.use_toeplitz(False):
        train(epoch)
        test()
    state_dict = model.state_dict()
    likelihood_state_dict = likelihood.state_dict()
    torch.save({'model': state_dict, 'likelihood': likelihood_state_dict}, 'dkl_cifar_checkpoint.dat')

/home/jrg365/gpytorch/gpytorch/models/abstract_variational_gp.py:34: UserWarning: model.marginal_log_likelihood is now deprecated. Please use gpytorch.mll.VariationalMarginalLogLikelihood instead.
  "model.marginal_log_likelihood is now deprecated. "


Train Epoch: 1 [001/782], Loss: 266.617157
Train Epoch: 1 [002/782], Loss: 245.251434
Train Epoch: 1 [003/782], Loss: 2.411211
Train Epoch: 1 [004/782], Loss: 2.421564
Train Epoch: 1 [005/782], Loss: 2.414730
Train Epoch: 1 [006/782], Loss: 2.398049
Train Epoch: 1 [007/782], Loss: 2.423582
Train Epoch: 1 [008/782], Loss: 2.452920
Train Epoch: 1 [009/782], Loss: 2.405890
Train Epoch: 1 [010/782], Loss: 2.437997
Train Epoch: 1 [011/782], Loss: 2.432441
Train Epoch: 1 [012/782], Loss: 2.417792
Train Epoch: 1 [013/782], Loss: 2.425106
Train Epoch: 1 [014/782], Loss: 2.442632
Train Epoch: 1 [015/782], Loss: 2.428365
Train Epoch: 1 [016/782], Loss: 2.448847
Train Epoch: 1 [017/782], Loss: 2.424292
Train Epoch: 1 [018/782], Loss: 2.478495
Train Epoch: 1 [019/782], Loss: 2.480617
Train Epoch: 1 [020/782], Loss: 2.468606
Train Epoch: 1 [021/782], Loss: 2.453479
Train Epoch: 1 [022/782], Loss: 2.453201
Train Epoch: 1 [023/782], Loss: 2.449346
Train Epoch: 1 [024/782], Loss: 2.474321
Train Epoch:

Train Epoch: 1 [201/782], Loss: 2.446805
Train Epoch: 1 [202/782], Loss: 2.432297
Train Epoch: 1 [203/782], Loss: 2.436419
Train Epoch: 1 [204/782], Loss: 2.401893
Train Epoch: 1 [205/782], Loss: 2.487541
Train Epoch: 1 [206/782], Loss: 2.434072
Train Epoch: 1 [207/782], Loss: 2.421194
Train Epoch: 1 [208/782], Loss: 2.420472
Train Epoch: 1 [209/782], Loss: 2.452245
Train Epoch: 1 [210/782], Loss: 2.443273
Train Epoch: 1 [211/782], Loss: 2.448035
Train Epoch: 1 [212/782], Loss: 2.385724
Train Epoch: 1 [213/782], Loss: 2.401109
Train Epoch: 1 [214/782], Loss: 2.431129
Train Epoch: 1 [215/782], Loss: 2.432420
Train Epoch: 1 [216/782], Loss: 2.407161
Train Epoch: 1 [217/782], Loss: 2.427823
Train Epoch: 1 [218/782], Loss: 2.414876
Train Epoch: 1 [219/782], Loss: 2.420785
Train Epoch: 1 [220/782], Loss: 2.391116
Train Epoch: 1 [221/782], Loss: 2.476599
Train Epoch: 1 [222/782], Loss: 2.430720
Train Epoch: 1 [223/782], Loss: 2.417209
Train Epoch: 1 [224/782], Loss: 2.429458
Train Epoch: 1 [

Train Epoch: 1 [401/782], Loss: 2.249115
Train Epoch: 1 [402/782], Loss: 2.281383
Train Epoch: 1 [403/782], Loss: 2.359003
Train Epoch: 1 [404/782], Loss: 2.349981
Train Epoch: 1 [405/782], Loss: 2.360611
Train Epoch: 1 [406/782], Loss: 2.403312
Train Epoch: 1 [407/782], Loss: 2.402438
Train Epoch: 1 [408/782], Loss: 2.351783
Train Epoch: 1 [409/782], Loss: 2.330976
Train Epoch: 1 [410/782], Loss: 2.395877
Train Epoch: 1 [411/782], Loss: 2.357258
Train Epoch: 1 [412/782], Loss: 2.316657
Train Epoch: 1 [413/782], Loss: 2.260732
Train Epoch: 1 [414/782], Loss: 2.258689
Train Epoch: 1 [415/782], Loss: 2.311068
Train Epoch: 1 [416/782], Loss: 2.223074
Train Epoch: 1 [417/782], Loss: 2.353785
Train Epoch: 1 [418/782], Loss: 2.200052
Train Epoch: 1 [419/782], Loss: 2.323462
Train Epoch: 1 [420/782], Loss: 2.322127
Train Epoch: 1 [421/782], Loss: 2.308682
Train Epoch: 1 [422/782], Loss: 2.269242
Train Epoch: 1 [423/782], Loss: 2.268649
Train Epoch: 1 [424/782], Loss: 2.259756
Train Epoch: 1 [

Train Epoch: 1 [601/782], Loss: 2.255654
Train Epoch: 1 [602/782], Loss: 2.325017
Train Epoch: 1 [603/782], Loss: 2.243802
Train Epoch: 1 [604/782], Loss: 2.185221
Train Epoch: 1 [605/782], Loss: 2.172384
Train Epoch: 1 [606/782], Loss: 2.292906
Train Epoch: 1 [607/782], Loss: 2.215364
Train Epoch: 1 [608/782], Loss: 2.199324
Train Epoch: 1 [609/782], Loss: 2.385331
Train Epoch: 1 [610/782], Loss: 2.272595
Train Epoch: 1 [611/782], Loss: 2.334003
Train Epoch: 1 [612/782], Loss: 2.141779
Train Epoch: 1 [613/782], Loss: 2.240621
Train Epoch: 1 [614/782], Loss: 2.287062
Train Epoch: 1 [615/782], Loss: 2.454718
Train Epoch: 1 [616/782], Loss: 2.278179
Train Epoch: 1 [617/782], Loss: 2.131039
Train Epoch: 1 [618/782], Loss: 2.174621
Train Epoch: 1 [619/782], Loss: 2.321344
Train Epoch: 1 [620/782], Loss: 2.196954
Train Epoch: 1 [621/782], Loss: 2.331423
Train Epoch: 1 [622/782], Loss: 2.118226
Train Epoch: 1 [623/782], Loss: 2.085387
Train Epoch: 1 [624/782], Loss: 2.389733
Train Epoch: 1 [